<a href="https://www.kaggle.com/code/ifeanyichukwunwobodo/an-analysis-of-factories-and-markets-in-lagos?scriptVersionId=130414404" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style='color: #223c5f ;
           background-color: #f1b04c;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: #FAF0E6;
           font-family: "Times Roman";'>
WHAT IS THE OPTIMAL FACTORY LOCATION IN LAGOS GIVEN THE DISTRIBUTION OF MARKET?
<a class="anchor" id="1"></a> 


# INTRODUCTION
In making business decisions, the location of plants and industries is a very important decison. This can be the main determinant of sales, cost, profit and losses. The location of plant and factories also determines the cost of transportation, speed of recovery and also whether a company enjoys the benefits of localization. Whether it is a new business or an old business just seeking expansion, most businesses pay considerable amount of attention and spend resource in selecting the right place to locate a factory.

Many factors are put into consideration in location of an industry, such as:

**Nearness to market**

**Nearness to competitors**

**Nearness to sources of inputs**

**Demographic factors**

**Macroeconomic and political factors (like the average income in the area, level of inflation in the area, political stability in the area, cultural differences  and so on)**

This analysis focuses on nearness to market due to data availability.

Why Lagos?
Lagos is the commercial capital of Nigeria and has the highest number of factories and seaports because of it's proximity to the Atlantic ocean. It is also the former capital of Nigeria and is a key target for businesses seeking to locate their firm and/factories. It also has the highest population of head offices for the various multinational and interstate companies in Nigeria.

# RESEARCH QUESTIONS

The general objective of the project is to determine the best place to locate a factory in Lagos

The project hopes to answer the following questions:

**What is the distribution of factories around Lagos?**

**What is the distribution of markets around Lagos?**

**Which part of Lagos has the densiest population of factories  and markets in Lagos?**

**What is the most efficient place to locate a factory given the distribution of market?**


# ABOUT THE DATA
This project aims at determining the best location of factories and industrial sites in Lagos using data retrieved from GRID3 A geographical database thaT???

In [1]:
import math #for basic maths calculation 
import numpy #for linear algebra
import pandas as pd # for data manipulation and basic data visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# for geographical analysis we use
import geopandas as gpd #To read our data 
import folium #for interactive maps
from folium import Choropleth, Circle, Marker #to select the maptype we want to use
from folium.plugins import HeatMap, MarkerCluster #for plugins
from pointpats import centrography
import warnings
warnings.simplefilter('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


For this we are going to use a geojson file obtained from GRID which contains geeogarphical data on the various factories and industrial sites (for the rest of this notebook, this will be reffered to simply as "Factories" in Lagos. Later we will introduce another dataset for markets.

In [2]:
factories = gpd.read_file('/kaggle/input/lagos-georeferenced-dataset/factoriesindustrial-sites.geojson')
#Let's take a look at the first few rows
factories.head()

,id,state_code,source,name,status,ward_code,global_id,geometry
0,u_fc_poi_factory_industry_site.1,LA,GRID,JJ International Block Industry,In Use,LASIUI05,000b39d0-ba96-4d62-91f3-52bef3d64890,POINT (3.78227 6.47244)
1,u_fc_poi_factory_industry_site.2,LA,GRID,Kadiri Block Industry,In Use,LASIUI04,001a5124-6f66-457f-ac1b-c4f1fde6a61a,POINT (3.68407 6.46090)
2,u_fc_poi_factory_industry_site.4,LA,GRID,Teju Blocks,In Use,LASIKU29,004d7b39-d3b8-4074-b270-d0fc09a20510,POINT (3.54362 6.59287)
3,u_fc_poi_factory_industry_site.11,LA,GRID,Adurumigba Block Industries,In Use,LASBDY08,006bb17c-b0fc-4b09-acc4-59250acdcfce,POINT (3.03417 6.46336)
4,u_fc_poi_factory_industry_site.18,LA,GRID,Abllat Nig Co Ltd,In Use,LASAMO24,0097ec77-7a51-44a4-a83d-c15f67e03fad,POINT (3.25955 6.52691)


In [3]:
factories.columns.to_list()

['id',
 'state_code',
 'source',
 'name',
 'status',
 'ward_code',
 'global_id',
 'geometry']

The datset contains data on the `factory id`, `state_code`, `name`, `ward_code`, `global_id` and `geometry`. Not all the columns are useful for analysis. Therefor, we keep only the important columns.

In [4]:
print( "The factories dataset contains", factories.shape[0], "rows and", factories.shape[1], "columns")

The factories dataset contains 732 rows and 8 columns


In [5]:
#Let's keep the
factories = factories[['name', 'status', 'geometry']].copy()

`name`: The name column shows the name of the factory.

`status`: This shows the status of the factory. The factories are categorized into three categories; 

* **In-use** : This represents factories that are still operational at the time the dataset was retrieved.

* **Unused-poor**: These are factories that are no longer in use and are in a poor state. 

* **Unused-good**: These are factories that are no longer in use and are in a good state.

`geometry`: This shows the location of the 


In [6]:
#Let's check for missing values
factories.isna().sum()

name        0
status      0
geometry    0
dtype: int64

There are no missing values in our dataset.

In [7]:
factories.describe().T

,count,unique,top,freq
name,732,718,Block Industry,7
status,732,3,In Use,701
geometry,732,732,POINT (3.7822694 6.4724391),1


In [8]:
print(factories.crs)


epsg:4326


?? Add the note from GAPF

Let's create Longitude ('lon') and Latitude ('lat') columns from the 'geometry' column. ??The YOU ADD WHAT IS THE GEOMETRY COLUMN AND WHAT IS LONGITUDE AND LATITUDE??

In [9]:
factories['lon'] = factories.geometry.apply(lambda p: p.x)
factories['lat'] = factories.geometry.apply(lambda p: p.y)

Most of the factories and industrial sites are still in use (701).


# EXPLORATORY ANALYSIS OF  THE MARKET DATASET 

Let's import the market dataset and have a look.

In [10]:
markets = gpd.read_file('/kaggle/input/lagos-georeferenced-dataset/markets.geojson')
markets.head()



,id,state_code,source,name,ward_code,settlement_name,settlement_type,type_goods,other_type_goods,product_desc,...,days_tuesday,days_wednesday,days_thursday,days_friday,days_saturday,days_sunday,total_days,frequency,global_id,geometry
0,u_fc_poi_market.51,LA,GRID,Ajebowale Market Ibowon,LASEPE13,Ibowon Market,None,Mixed,None,None,...,True,True,True,True,True,True,7.0,Other,0104c14b-b08c-41d6-8912-fcb08495ab43,POINT (3.99884 6.67332)
1,u_fc_poi_market.55,LA,OSGOF,Alasela Market,LASSMU12,Alasela,BUA,None,None,None,...,None,None,None,None,None,None,NaN,None,0133d84c-f511-4fd1-98e2-5a81765898b2,POINT (3.36763 6.53302)
2,u_fc_poi_market.84,LA,GRID,Trebet Superstore,LASEOA10,Peninsula,None,Clothing Food Stuff Mixed,None,None,...,True,True,True,True,True,True,7.0,Weekly,01e74f2e-4e2a-49ed-9727-940868c6a13c,POINT (3.47132 6.44261)
3,u_fc_poi_market.87,LA,OSGOF,Lagos International Trade Fair Complex,LASAON01,Lagos International Trade Fair Complex,BUA,None,None,None,...,None,None,None,None,None,None,NaN,None,01f199f4-2e69-443b-b5bd-9c37bd65aeeb,POINT (3.24626 6.45804)
4,u_fc_poi_market.88,LA,GRID,Lsdpc Shoping Complex,LASIOE10,Ogba Ijaiye,None,Clothing Food Stuff Mixed,None,None,...,True,False,False,False,False,False,1.0,Weekly,01f3c85f-d6f1-4374-a08a-d7c625c12b49,POINT (3.33240 6.62796)


In [11]:
print('The market dataset has', markets.shape[0], 'and' , markets.shape[1], 'columns')

The market dataset has 498 and 21 columns


In [12]:
markets.columns


Index(['id', 'state_code', 'source', 'name', 'ward_code', 'settlement_name',
       'settlement_type', 'type_goods', 'other_type_goods', 'product_desc',
       'days_monday', 'days_tuesday', 'days_wednesday', 'days_thursday',
       'days_friday', 'days_saturday', 'days_sunday', 'total_days',
       'frequency', 'global_id', 'geometry'],
      dtype='object')

In [13]:
# Create longitude and latitude columns  columns for the market dataset
markets['lon'] = markets.geometry.apply(lambda p: p.x)
markets['lat'] = markets.geometry.apply(lambda p: p.y)

In [14]:
px.sunburst(markets, values='frequency', parents='type_goods', color_continuous_scale='blues')

In [15]:
status = factories.groupby('status').count().reset_index() #To get the count of the different categories

px.bar(status, 
       color= 'status',
       color_discrete_map={'In Use': 'blue', 'Unused Good':'yellow', 'Unused Poor': 'purple'},
       title='Status of Factories and Indistrial Sites in Lagos') 

# **What is the distribution of factories and Markets around Lagos?**

To answer this, we use the scatter map and bubble map. The scatter map will show the locations of the different factories/industrial sites in Lagos. On the other hand, the bubble map will show the count of the various factories in Lagos

## Distribution of factories in Lagos

In [16]:

fig = px.scatter_mapbox(factories,lat='lat', lon='lon', zoom=10, color='status',
                           mapbox_style="carto-positron", hover_name='name',
                        color_discrete_map={'In Use': 'steelblue', 'Unused Good': 'red', 'Unused Poor': 'green'})
fig.update_layout(margin=dict(l=0, r=0, t=30, b=10))

fig.show()

As explained in the bar chart earlier, the number of number of unused factories (both poor and good) are scanty relative to factories still in use. An important thing to note is that the unused factories are in the busy areas of Lagos (Ikorodu and Ikeja). This can be interpreted to mean that the factories are out of service because they are unable to keep up with competition and/or increase cost of production in these areas (the increase in the cost of rent that comes from the localization of industries).

Let's investigate the number of factories in different areas in Lagos.

In [17]:
bubble = folium.Map(location =[6.5244, 3.3792], 
                       zoom_start =9,tiles ='cartodbpositron')

mc = MarkerCluster()
for idx, row in factories.iterrows():
    mc.add_child(Marker([row['lat'], row['lon']]))
bubble.add_child(mc)

bubble

In the map above, the points on the map with a only one factory are represented by markers. The green colored bubble shows when data point is less than 10. The yellow shows the factories are greater than 10. This is not the color scheme in all situations. The chart shows that there are a lot of factories around Ikeja and Ikorodu (as shown by the yellow bubbles). There are a lot of factories in Ikorodu because of the proximity to Lagos-Ibadan expressway which leads to ease in transportation, availability of water because of the Lagos Lagoon, access to electricity as Ikorpdu has a relatively greater access to power than other oarts of Lagos and availability of land. Ikeja on the other 

The map also shows that as you near the edges of Lagos, the number of factories begins to decrease. The factories are also scanty in the southern part of the Lagos Lagoon with 139  factories in total.


## Distribution of Markets in Lagos

In [18]:

fig = px.scatter_mapbox(markets,lat='lat', lon='lon', zoom=10, color='frequency',
                           mapbox_style="carto-positron", hover_name='name',
                       # color='type_goods'
                       )
fig.update_layout(margin=dict(l=0, r=0, t=30, b=10))

fig.show()

# **Which part of Lagos is the most competitive place to locate a factory and/or market?**



Answering this question will be instrumental in decisions that involve locating factory in a position that will allow th firm to enjoy the benefits of localization and also location that where the firm will have access to the most markets. It is also instrumental to firms who wants too achieve the opposite.

In [19]:
# mean center for factories
mc = centrography.mean_center(factories[["lon", "lat"]])
medc = centrography.euclidean_median(factories[["lon", "lat"]])

# mean center for markets
mc1 = centrography.mean_center(markets[["lon", "lat"]])
medc1 = centrography.euclidean_median(markets[["lon", "lat"]])

# Scatter map of markets
fig = go.Figure(go.Scattermapbox(
    lat=markets["lat"],
    lon=markets["lon"],
    name='Markets',
    hovertext=markets['name'],
    mode="markers",
    marker=dict(
        size=5,
        color="orange",
    ),
))

# mean center of markets
fig.add_trace(go.Scattermapbox(
    lat=[mc1[1]],
    lon=[mc1[0]],
    mode="markers",
    name = "Mean Center of Markets in Lagos",
    marker=dict(
        size=10,
        color="yellow",
    ),
    text=["Mean Center"],
))

# median center of markets
fig.add_trace(go.Scattermapbox(
    lat=[medc1[1]],
    lon=[medc1[0]],
    name = "Median Center of Markets in Lagos",
    mode="markers",
    marker=dict(
        size=10,
        color="magenta"
    ),
    text=["Median Center"],
))


# Mean center i
fig.add_trace(go.Scattermapbox(
    lat=[mc[1]],
    lon=[mc[0]],
    mode="markers",
    name = "Mean Center of Factories",
    marker=dict(
        size=10,
        color="red",
    ),
    text=["Mean Center"],
))
fig.add_trace(go.Scattermapbox(
    lat=[medc[1]],
    lon=[medc[0]],
    name = "Median Center of Factories",
    mode="markers",
    marker=dict(
        size=10,
        color="green"
    ),
    text=["Median Center"],
))

# Add a marker for the mean center
fig.add_trace(go.Scattermapbox(
    lat=factories["lat"],
    lon=factories["lon"],
    name='Factories/Industrial Sites',
    hovertext=factories['name'],
    mode="markers",
    marker=dict(
        size=5,
        color="steelblue",
    ),
))
# Set the layout of the plot
layout = go.Layout(
    mapbox=dict(
        center=dict(lat=6.5244, lon= 3.3792),
        style="carto-positron",
        zoom=10
    ),
)

# Update the figure with the layout
fig.update_layout(layout)

# Show the plot
fig.show()


The markets are clustered around Ikeja near the Lagos Lagoon while the factories are clustered at the opposite side of the Lagos Lagoon at Ikorodu. This is because of the various seaports that are loacted near the lagoon. The discrepancy between the two centers is caused by the skew. Thus,the far out clusters of factories and markets pulls the mean center to the east, relative to the median center for the factories and west relative too the median center for the market.

# **What is the least competitive place to locate a factory given the distribution of market?**

As mentioned earleir, the reason why some factories were put out of use was likely because of excessive competition. A new firm/factory can easily be driven out of market in a very competitive market so it is wiser to locate firm in a position with least competition but with access to market with customers that have easier access.

In [20]:
coverage = factories.geometry.buffer(0.01)
my_union = coverage.geometry.unary_union
outside_range = markets.loc[~markets["geometry"].apply(lambda x: my_union.contains(x))]

In [21]:
percentage = round(100*len(outside_range)/len(markets), 2)
print(f'Percentage of markets farther than 10km from any factory: {percentage}')

Percentage of markets farther than 10km from any factory: 38.35


38% of markets are located farther than 10km from the  factories. An organisation can take advantage of this and locate factory in a position that is nearest to the most amount of markets in order to reduce transportation cost and increase speed of delivery.

In [22]:
locate = folium.Map(location=[6.5244, 3.3792], zoom_start=11, tiles ='cartodbpositron') 

coverage = gpd.GeoDataFrame(geometry=factories.geometry).buffer(0.01)
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(locate)
HeatMap(data=outside_range[['lat', 'lon']], radius=9).add_to(locate)
folium.LatLngPopup().add_to(locate)
locate

From the map above, the blue circle represent 10km radius around each factory. Underneath it is a heatmap of the markets that are outsude the 10km range. The part of the heatmap that are still visible ouside the buffers are good locations to capture the markets that are too far from any factory. But an optimal location will be at 6.4550 latitude and 3.3978 longitude. Although many factories are located near this point, many markets are far from these factories and this location is also close to the Lagos Lagoon.

<div style='color: #223c5f ;
           background-color: #f1b04c;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: #FAF0E6;
           font-family: "Times Roman";'>
KEY FINDINGS
<a class="anchor" id="1"></a> 

    1. There are more factories in use than unused in Lagos.
    
    2.  The unused factories were likely pushed out by competition.
    
    3. Most factories and markets are located around the Lagos Lagoon because of proximity to seaports, historical factors such as the availability of lands in these areas and transportation advantages as a result nearness to major roads, seaport and markets. The same holds for markets but in this case it is nearness to factories.
    
    4. The least competitive place to locate your firm is near Lekki.

# LIMITATION OF THE ANALYSIS

Because of data constraint and to allow the analysis to be meaningful, some assumptions were made:
    
* The analysis assumes that the factories only supply to the market: This is not always true as some of the markets are market for input. On another note, the factories also supply to each other as output in one industry is used as input in another. 

* The inustry/sector of the market and factories were not put into consideration: the various factories exists in seperate markets an =d do not always have to fight for the same consumer. 

*  All the factories compete with each other: This is also related to the second assumption.A factory can enjoy a competitive advantage by locating itself among other non-competiting factories and competitive ones. This was not taken into account as the data did not give any idea on the sector (as mentioned earlier).

 


<div style='color: #223c5f ;
           background-color: #f1b04c;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: #FAF0E6;
           font-family: "Times Roman";'>
CONCLUSION
<a class="anchor" id="1"></a> 
    
    Even with the various constraints and assumptions, the analysis is still important as it gave a general idea on distribution, sparsity and density of factories and markets in Lagos. This can inform both private and public investors decision concerning the location of factories and markets in Lagos. Another benefit is that the maps created as a result of this analysis are interactive in nature. Hence, they also serve as factories and markets tracker.


    That will be all for today. Thank you for reading to the end! Remember to leave an upvote if you found this notebook helpful to allow others to discover it and also remember to drop a comment if you have any contribution or question. Thank you!